In [1]:
import torch
import pickle
from tqdm import tqdm
import copy
import pandas as pd
import csv

In [2]:
infer_category = [12, 10, 2, 6, 13, 15, 18, 19, 21, 22, 25, 26, 36, 37, 39, 48]
LR = 2e-6
model_name = '470000'
exp_folder = f'exp_0.001_final_test_16'
device = 'cuda:0'

In [3]:
# cnn_model
# class MyModel(torch.nn.Module):
#     def __init__(self):
#         super(MyModel, self).__init__()
#         # self.id_encoder = torch.nn.Embedding(500000, 105)
#         self.conv = torch.nn.Conv1d(16, 16, 7, padding='same')
#         self.encoder1 = torch.nn.Linear(23, 128)
#         self.dropout1 = torch.nn.Dropout(p = 0.1)
#         self.relu1 = torch.nn.ReLU()
#         self.decoder1 = torch.nn.Linear(128, 256)
#         self.dropout5 = torch.nn.Dropout(p = 0.1)
#         self.relu5 = torch.nn.ReLU()
#         self.decoder3 = torch.nn.Linear(256, 64)
#         self.dropout7 = torch.nn.Dropout(p = 0.1)
#         self.relu7 = torch.nn.ReLU()
#         self.decoder4 = torch.nn.Linear(64, 16)
#         self.dropout8 = torch.nn.Dropout(p = 0.1)
#         self.relu8 = torch.nn.ReLU()
#         self.decoder5 = torch.nn.Linear(16, 1)

#     def forward(self, x, y):
#         # y = self.id_encoder(y)
#         # y = y.squeeze()

#         x = self.conv(x)
        
#         # x = torch.cat((x, y), 2)
#         x = self.encoder1(x)
#         x = self.dropout1(x)
#         x = self.relu1(x)
    
#         x = self.decoder1(x)
#         x = self.dropout5(x)
#         x = self.relu5(x)

#         x = self.decoder3(x)
#         x = self.dropout7(x)
#         x = self.relu7(x)

#         x = self.decoder4(x)
#         x = self.dropout8(x)
#         x = self.relu8(x)
        
#         x = self.decoder5(x)
        
#         return torch.nn.functional.sigmoid(x)

In [4]:
# # avgpooling or maxpooling and concat and 
# class MyModel(torch.nn.Module):
#     def __init__(self):
#         super(MyModel, self).__init__()
#         # self.id_encoder = torch.nn.Embedding(500000, 105)
#         self.conv1 = torch.nn.Conv1d(16, 16, 7, padding='same')
#         # self.maxpooling = torch.nn.AvgPool1d(kernel_size=3, stride=1, padding=1)
#         # self.avgpooling = torch.nn.AvgPool1d(kernel_size=3, stride=1, padding=1)
#         self.conv2 = torch.nn.Conv1d(16, 16, 7, padding='same')
#         self.conv3 = torch.nn.Conv1d(16, 16, 7, padding='same')
#         self.encoder1 = torch.nn.Linear(23 * 3, 128)
#         self.dropout1 = torch.nn.Dropout(p = 0.1)
#         self.relu1 = torch.nn.ReLU()
#         self.decoder1 = torch.nn.Linear(128, 256)
#         self.dropout5 = torch.nn.Dropout(p = 0.1)
#         self.relu5 = torch.nn.ReLU()
#         self.decoder3 = torch.nn.Linear(256, 64)
#         self.dropout7 = torch.nn.Dropout(p = 0.1)
#         self.relu7 = torch.nn.ReLU()
#         self.decoder4 = torch.nn.Linear(64, 16)
#         self.dropout8 = torch.nn.Dropout(p = 0.1)
#         self.relu8 = torch.nn.ReLU()
#         self.decoder5 = torch.nn.Linear(16, 1)

#     def forward(self, x):
#         # y = self.id_encoder(y)
#         # y = y.squeeze()

#         a = self.conv1(x)
#         # a = self.avgpooling(a)
#         b = self.conv2(x)
#         # b = self.avgpooling(b)
#         c = self.conv3(x)    
#         # c = self.avgpooling(c)
        
#         x = torch.cat((a, b, c), dim=2)
#         x = self.encoder1(x)
#         x = self.dropout1(x)
#         x = self.relu1(x)
    
#         x = self.decoder1(x)
#         x = self.dropout5(x)
#         x = self.relu5(x)

#         x = self.decoder3(x)
#         x = self.dropout7(x)
#         x = self.relu7(x)

#         x = self.decoder4(x)
#         x = self.dropout8(x)
#         x = self.relu8(x)
        
#         x = self.decoder5(x)
        
#         return torch.sigmoid(x)

In [5]:
# # cnn and lstm 
# class MyModel(torch.nn.Module):
#     def __init__(self):
#         super(MyModel, self).__init__()
#         # self.id_encoder = torch.nn.Embedding(500000, 105)
#         self.conv1 = torch.nn.Conv1d(16, 16, 7, padding='same')
#         # self.maxpooling = torch.nn.MaxPool1d(kernel_size=5, padding=2, stride=1)
#         self.conv2 = torch.nn.Conv1d(16, 16, 7, padding='same')
#         self.conv3 = torch.nn.Conv1d(16, 16, 7, padding='same')
#         self.lstm = torch.nn.LSTM(
#             input_size = 23,
#             hidden_size = 59,
#             num_layers = 3,
#             batch_first = True,
#             dropout = 0.1,
#             bidirectional = False,
#         )
#         self.encoder1 = torch.nn.Linear(128, 128)
#         self.dropout1 = torch.nn.Dropout(p = 0.1)
#         self.relu1 = torch.nn.ReLU()
#         self.decoder1 = torch.nn.Linear(128, 256)
#         self.dropout5 = torch.nn.Dropout(p = 0.1)
#         self.relu5 = torch.nn.ReLU()
#         self.decoder3 = torch.nn.Linear(256, 64)
#         self.dropout7 = torch.nn.Dropout(p = 0.1)
#         self.relu7 = torch.nn.ReLU()
#         self.decoder4 = torch.nn.Linear(64, 16)
#         self.dropout8 = torch.nn.Dropout(p = 0.1)
#         self.relu8 = torch.nn.ReLU()
#         self.decoder5 = torch.nn.Linear(16, 1)

#     def forward(self, x):

#         a = self.conv1(x)
#         b = self.conv2(x)
#         c = self.conv3(x)
#         lstm_output, _ = self.lstm(x)

#         x = torch.cat((a, b, c, lstm_output), dim = 2)
#         x = self.encoder1(x)
#         x = self.dropout1(x)
#         x = self.relu1(x)
    
#         x = self.decoder1(x)
#         x = self.dropout5(x)
#         x = self.relu5(x)

#         x = self.decoder3(x)
#         x = self.dropout7(x)
#         x = self.relu7(x)

#         x = self.decoder4(x)
#         x = self.dropout8(x)
#         x = self.relu8(x)
        
#         x = self.decoder5(x)
        
#         return torch.sigmoid(x)

In [8]:
class MyModel(torch.nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.hidden_size = 7
        self.input_size = 1
        # self.embedding_size = 5
        # self.id_encoder = torch.nn.Embedding(16, self.embedding_size)
        self.conv1 = torch.nn.Conv1d(16, 16, 7, padding=0)
        # self.maxpooling = torch.nn.MaxPool1d(kernel_size=5, padding=2, stride=1)
        self.conv2 = torch.nn.Conv1d(16, 16, 7, padding=0)
        self.conv3 = torch.nn.Conv1d(16, 16, 7, padding=0)
        self.lstm = torch.nn.LSTM(
            input_size = self.input_size,
            hidden_size = self.hidden_size,
            num_layers = 4,
            batch_first = True,
            dropout = 0.1,
            bidirectional = False,
        )
        self.encoder_origin_1 = torch.nn.Linear(23, 64)
        self.encoder_cnn_1 = torch.nn.Linear(17 * 3, 128)
        self.encoder_lstm_1 = torch.nn.Linear(self.hidden_size, 32)

        self.encoder_origin_2 = torch.nn.Linear(64, 256)
        self.encoder_cnn_2 =  torch.nn.Linear(128, 512)
        self.encoder_lstm_2 = torch.nn.Linear(32, 128)

        self.relu = torch.nn.ReLU()
        self.dropout = torch.nn.Dropout(p = 0.1)

        self.feedforward_origin_1 = torch.nn.Linear(256, 256)
        self.feedforward_cnn_1 = torch.nn.Linear(512, 512)
        self.feedforward_lstm_1 = torch.nn.Linear(128, 128)

        self.decoder_origin_1 = torch.nn.Linear(256, 64)
        self.decoder_cnn_1 = torch.nn.Linear(512, 128)
        self.decoder_lstm_1 = torch.nn.Linear(128, 32)

        self.decoder_origin_2 = torch.nn.Linear(64, 16)
        self.decoder_cnn_2 = torch.nn.Linear(128, 32)
        self.decoder_lstm_2 = torch.nn.Linear(32, 8)

        self.decoder_origin_3 = torch.nn.Linear(16, 4)
        self.decoder_cnn_3 = torch.nn.Linear(32, 8)
        self.decoder_lstm_3 = torch.nn.Linear(8, 2)

        self.predict = torch.nn.Linear(14, 1)


    def forward(self, x):
        batch = x.size()[0]

        a = self.conv1(x)
        b = self.conv2(x)
        c = self.conv3(x)
        d = x.unsqueeze(dim = 3)

        tmp_list = []
        
        for i in range(batch):
            tmp_list.append(torch.unsqueeze(torch.squeeze(self.lstm(d[i])[0][ :, -1:, :], dim=1), dim = 0))

        lstm_output = torch.cat(tmp_list, dim = 0)
        cnn = torch.cat((a, b, c), dim = 2)

        x = self.encoder_origin_1(x)
        cnn = self.encoder_cnn_1(cnn)
        lstm_output = self.encoder_lstm_1(lstm_output)

        x = self.encoder_origin_2(x)
        x = self.relu(x)
        x = self.dropout(x)
        cnn = self.encoder_cnn_2(cnn)
        cnn = self.relu(cnn)
        cnn = self.dropout(cnn)
        lstm_output = self.encoder_lstm_2(lstm_output)
        lstm_output = self.relu(lstm_output)
        lstm_output = self.dropout(lstm_output)

        x = self.feedforward_origin_1(x)
        x = self.relu(x)
        x = self.dropout(x)
        cnn = self.feedforward_cnn_1(cnn)
        cnn = self.relu(cnn)
        cnn = self.dropout(cnn)
        lstm_output = self.feedforward_lstm_1(lstm_output)
        lstm_output = self.relu(lstm_output)
        lstm_output = self.dropout(lstm_output)

        x = self.decoder_origin_1(x)
        x = self.relu(x)
        x = self.dropout(x)
        cnn = self.decoder_cnn_1(cnn)
        cnn = self.relu(cnn)
        cnn = self.dropout(cnn)
        lstm_output = self.decoder_lstm_1(lstm_output)
        lstm_output = self.relu(lstm_output)
        lstm_output = self.dropout(lstm_output)

        x = self.decoder_origin_2(x)
        x = self.relu(x)
        x = self.dropout(x)
        cnn = self.decoder_cnn_2(cnn)
        cnn = self.relu(cnn)
        cnn = self.dropout(cnn)
        lstm_output = self.decoder_lstm_2(lstm_output)
        lstm_output = self.relu(lstm_output)
        lstm_output = self.dropout(lstm_output)

        x = self.decoder_origin_3(x)
        x = self.relu(x)
        x = self.dropout(x)
        cnn = self.decoder_cnn_3(cnn)
        cnn = self.relu(cnn)
        cnn = self.dropout(cnn)
        lstm_output = self.decoder_lstm_3(lstm_output)
        lstm_output = self.relu(lstm_output)
        lstm_output = self.dropout(lstm_output)

        for_predict = torch.cat((x, cnn, lstm_output), dim=2)
        x = self.predict(for_predict)

        return torch.sigmoid(x)


In [9]:
# loading the model
torch.no_grad()
model = MyModel()
model.load_state_dict(torch.load(f'./{exp_folder}/{model_name}.model', map_location='cpu'))
model = model.to(device)
model.eval()

MyModel(
  (conv1): Conv1d(16, 16, kernel_size=(7,), stride=(1,))
  (conv2): Conv1d(16, 16, kernel_size=(7,), stride=(1,))
  (conv3): Conv1d(16, 16, kernel_size=(7,), stride=(1,))
  (lstm): LSTM(1, 7, num_layers=4, batch_first=True, dropout=0.1)
  (encoder_origin_1): Linear(in_features=23, out_features=64, bias=True)
  (encoder_cnn_1): Linear(in_features=51, out_features=128, bias=True)
  (encoder_lstm_1): Linear(in_features=7, out_features=32, bias=True)
  (encoder_origin_2): Linear(in_features=64, out_features=256, bias=True)
  (encoder_cnn_2): Linear(in_features=128, out_features=512, bias=True)
  (encoder_lstm_2): Linear(in_features=32, out_features=128, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.1, inplace=False)
  (feedforward_origin_1): Linear(in_features=256, out_features=256, bias=True)
  (feedforward_cnn_1): Linear(in_features=512, out_features=512, bias=True)
  (feedforward_lstm_1): Linear(in_features=128, out_features=128, bias=True)
  (decoder_origin_1): Linear(i

In [10]:
# read the file to inference
f = open('dataset_for_infer.pickle', 'rb')
data = pickle.load(f)
f.close()

In [11]:
# using the model to inference the output
for i in tqdm(data.keys(), desc = 'tensor'):
    # for itr in range(len(data[i])):
    #     data[i][itr] = data[i][itr][12:]
    data[i] = torch.Tensor(data[i])

infer_output = {}

for i in tqdm(data.keys(), desc='doing inference'):

    tmp = data[i].unsqueeze(dim =0)
    tmp = tmp.to(device)

    tmp = model(tmp).squeeze().tolist()
    # using for no user_id
    infer_output[str(i)] = tmp

    # infer_output[str(i)] = model(data[i].unsqueeze(dim =0),1).squeeze().tolist()

for i in tqdm(infer_output.keys(), desc='create tuple'):
    infer_output[i] = list(infer_output[i])

for i in tqdm(infer_output.keys(), desc = 'zipping item'):
    infer_output[i] = list(zip(infer_category, infer_output[i]))

for i in tqdm(infer_output.keys(), desc = 'sorting'):
    infer_output[i].sort(key = lambda tup : tup[1], reverse = True)

for i in tqdm(infer_output.keys(), desc = 'spliting'):
    infer_output[i] = infer_output[i][:3]

for i in tqdm(infer_output.keys(), desc = 'picking'):
    for itr1 in range(len(infer_output[i])):
        infer_output[i][itr1] = infer_output[i][itr1][0]

print(infer_output['10000000'])

picking: 100%|██████████| 498040/498040 [00:00<00:00, 715215.19it/s]

[10, 48, 12]


In [ ]:
infer_output['10000000']

[10, 48, 12]

In [12]:
# creating csv file
df = pd.DataFrame.from_dict(infer_output)
df = pd.DataFrame.transpose(df)
final_infer = pd.DataFrame.to_csv(df)

f = open(f'./infer_output/{model_name}.csv', 'w')
f.write(final_infer)
f.close()
